In [21]:
import backtrader as bt
import pandas as pd
import numpy as np
# from get_score import get_score_ch
from sqlalchemy import create_engine
import talib as ta
import datetime
import pandas as pd

In [22]:
engine= create_engine('sqlite:///../../dataset/us/us_ticker_price_yf.db')

In [25]:
ticker_name = 'NVDA'
df = pd.read_sql('{}'.format(ticker_name), engine)
df.rename(columns={'hgih':'high'},inplace=True)
# df['chg_pct'] = ((df.close - df.open)/df.open)*100
df['close'] = df['close'].apply(lambda x: round(x,3))
df['VMA20'] = 0
df['VMA20'] = ta.SMA(df.volume, timeperiod=20)
df

,open,high,low,close,Adj Close,volume,datetime,VMA20
0,0.437500,0.488281,0.388021,0.410,0.376630,271468800.0,1999-01-22,NaN
1,0.442708,0.458333,0.410156,0.453,0.416086,51048000.0,1999-01-25,NaN
2,0.458333,0.467448,0.411458,0.418,0.383804,34320000.0,1999-01-26,NaN
3,0.419271,0.429688,0.395833,0.417,0.382609,24436800.0,1999-01-27,NaN
4,0.416667,0.419271,0.412760,0.415,0.381413,22752000.0,1999-01-28,NaN
...,...,...,...,...,...,...,...,...
5904,154.300003,160.369995,153.889999,158.380,158.380005,46763500.0,2022-07-08,NaN
5905,155.259995,155.270004,150.380005,151.520,151.520004,43732100.0,2022-07-11,NaN
5906,152.679993,154.309998,148.770004,150.820,150.820007,45848300.0,2022-07-12,NaN
5907,145.899994,153.750000,144.649994,151.640,151.639999,52191800.0,2022-07-13,NaN


In [17]:
# make sure target date
target_date = '2021-10-19'
target_index = df[df.datetime == target_date].index.values[0] + 1
# 
time_zone = 60
start_index = target_index - time_zone
window = df[start_index : target_index]
window.index = range(len(window))
price_max = window.high.max()
index_max = window[window.high == price_max].index.values[0]
price_min = window[index_max:].low.min()
index_min = window[window.low == price_min].index.values[0]
if (price_max - price_min) / price_max < 0.3:
    print('fine')
if window.tail(1).close.values[0] < (price_max - price_min)/3+ price_min and window.tail(3).volume.mean() < window.tail(3).VMA20.mean():
    print('cheat zone')
if window.tail(1).close.values[0] > price_max - (price_max - price_min)/3 and window.tail(3).volume.mean() < window.tail(3).VMA20.mean():
    print('handle zone')

fine


In [18]:
window.tail(3).volume.mean(), window.tail(3).VMA20.mean()

(19265366.666666668, nan)

In [8]:
window.tail(1).close

59    222.9
Name: close, dtype: float64

In [6]:
index_max, price_max,index_min, price_min, 

(24, 230.42999267578125, 48, 195.5500030517578)

(56, 44.79999923706055)

fine
